In [92]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests, sys, io
import pandas as pd

# Scraping

In [2]:
nasa_news_url="https://mars.nasa.gov/news/"

In [8]:
browser = Browser('chrome', headless=True)
browser.visit(nasa_news_url)

In [9]:
html = browser.html

In [10]:
soup=bs(html,'lxml')

In [11]:
latest_list_item = soup.find("div", class_="list_text")

In [12]:
news_title = latest_list_item.find("div", class_="content_title").text
news_p = latest_list_item.find("div", class_="article_teaser_body").text

print(f"{news_title} : {news_p}")

Robotic Toolkit Added to NASA's Mars 2020 Rover : The bit carousel, which lies at the heart of the rover's Sample Caching System, is now aboard NASA's newest rover. 


# JPL Mars Space Images - Featured Image

In [24]:
## https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars
base_url= "https://www.jpl.nasa.gov"
mars_images_link = f"{base_url}/spaceimages/?search=&category=Mars"

In [25]:
browser = Browser('chrome', headless=True)
browser.visit(mars_images_link)

In [26]:
html = browser.html

In [27]:
soup=bs(html,'lxml')

In [28]:
result = soup.find("footer")

In [29]:
relative_img_url=result.find("a")["data-link"]
print(relative_img_url)

/spaceimages/details.php?id=PIA00063


In [30]:
data_link = f"{base_url}{relative_img_url}"
print (data_link)

https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA00063


In [31]:
#browser = Browser('chrome', headless = True)
browser.visit(data_link)

In [32]:
html = browser.html
soup = bs(html, 'lxml')

In [33]:
hires_image_url=soup.find("article").find("figure", class_="lede").find("a")["href"]
# /spaceimages/images/largesize/PIA17175_hires.jpg


In [34]:
featured_image_url = f"{base_url}{hires_image_url}"
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA00063_hires.jpg


# Mars Weather

In [35]:
twitter_link = "https://twitter.com/marswxreport?lang=en"

In [36]:
response = requests.get(twitter_link)

In [37]:
print(response)

<Response [200]>


In [41]:
soup = bs(response.text, 'lxml')

In [79]:
results = soup.find_all('div', class_="content")
mars_weather=""

In [80]:
for result in results:
    tweet_text = result.find("p", class_ = "tweet-text")
    tweet_link = result.find("a", class_ = "twitter-timeline-link")
    tweet_date = result.find("span", class_ = "_timestamp")
    if "InSight sol" in result.text:
        #print("~" * 100)
        if tweet_link:
            #print(f"[{tweet_date.text}] => {tweet_text.text} \n{tweet_link.text}")
            #print("Modified String")
            #print(tweet_text.text.replace(tweet_link.text,""))
            mars_weather=tweet_text.text.replace(tweet_link.text,"")
            break
        else:
            print(f"[{tweet_date.text}] => {tweet_text.text}")
            break
print(mars_weather)

InSight sol 253 (2019-08-13) low -100.0ºC (-148.1ºF) high -25.5ºC (-13.9ºF)
winds from the SSE at 4.6 m/s (10.3 mph) gusting to 16.4 m/s (36.6 mph)
pressure at 7.60 hPa


# Mars Facts

In [81]:
mars_facts_url = "https://space-facts.com/mars/"

In [84]:
tables = pd.read_html(mars_facts_url)
print(type(tables))

<class 'list'>


In [91]:
df=tables[1]
df.rename(columns={0:"Mars Statistic Type", 1:"Mars Statistic Value"}, inplace = True)
df.to_html()

,Mars Statistic Type,Mars Statistic Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [97]:
html_buf=io.StringIO()

In [98]:
df.to_html(buf=html_buf, classes="table table-bordered table-hover", table_id="id_table_data", justify="inherit")

In [99]:
data_html_fp=open("mars_statistics_data.html","w")
data_html_fp.write(html_buf.getvalue())
data_html_fp.close()

# Mars Hemispheres